# Tópicos

1. Automatic Differentiation with ```torch.autograd```


2. Tensores, Funções e Grafos


3. Cálculo de Gradientes

4. Desabilitando o Rastreamento do Gradiente

5. Mais sobre grafos computacionais


# Automatic Differentiation with ```torch.autograd```

Em treinos de redes neurais, o algoritmo mais utilizado é o **back propagation**. Neste algoritmo, paramêtros são ajustados de acordo com o **gradiente** da função de perda.

Para computar estes gradientes, PyTorch tem uma ferramente de diferenciação chamada ```torch.autograd```.

Considerando uma rede de uma camada, de entrada **X**, paramêtros **w** e **b** e alguma função de perda.
Da forma:

In [6]:
import torch

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

x = torch.ones(5) #Input
y = torch.zeros(3) #Target
w = torch.randn(5, 3, requires_grad=True) #Weights
b = torch.randn(3, requires_grad=True) #Bias
z = torch.matmul(x, w) + b #Linear function
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y) #Loss function

Using cuda device


# Tensores, Funções e Grafos

O código acima define a seguinte função:

![Teste](images/computational_graph.png)

Nesta rede, **w**, **b** são parâmetros a serem otimizados. Então, é necessário computar gradientes da função de perda com as respectivas variáveis. Para isso, a definição ```requires_grad``` para os tensores é feita.

A função aplicada aos tensores para construção dos grafos é um objeto da classe ```function```. Este objeto computa a função da direção definida em ```forward``` e computar o ```backward propagation```. Uma referência a função backward propagation está em ```grad_fn```.

In [7]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x0000024857CE4310>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x0000024857CE5120>


# Cálculo de Gradientes

Para otimizar os pesos de uma rede neural, as derivadas da função de perda precisam ser calculadas, da forma $$ \frac{\partial loss}{\partial w} $$ e $$ \frac{\partial loss}{\partial b} $$

O cálculo dessas derivadas é feito com ```loss.backward()``` e, en seguida, recuperar ```w.grad```e ```b.grad```.

In [8]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0371, 0.0384, 0.0362],
        [0.0371, 0.0384, 0.0362],
        [0.0371, 0.0384, 0.0362],
        [0.0371, 0.0384, 0.0362],
        [0.0371, 0.0384, 0.0362]])
tensor([0.0371, 0.0384, 0.0362])


# Desabilitando o Rastreamento do Gradiente

Por padrão, todos tensores ```requires_grad=True``` rastreiam o histórico computacional e fazerm o cálculo do gradiente. Há alguns casos em que não é necessário fazer isso. Por exemplo, quando um modelo é treinado e ele deve ser aplicado a apenas alguns dados de entrada, ou seja, apenas cálculos progessivos pela rede. Para interromper, faz-se ```torch.no_grad()```.

In [9]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Desabilitar o rastreamento do gradiente quando:

* Marcar parâmetros na rede como **parâmetros congelados**
* **Acelerar os cálculos** quando a necessidade é fazer a passagem para frente, pois cálculos em tensores que não rastreiam gradientes são mais eficientes.

# Mais sobre grafos computacionais

O autograd mantém um registro dos dados (tensores) e de todas as operações executadas (juntamente com os novos tensores resultantes) em um grafo acíclico direcionado (DAG) composto por objetos <u>Function</u>.

Neste DAG, as folhas são os tensores de entrada e as raízes são os tensores de sáida. Ao traçar este grafo das raízes às folhas, calcula-se automaticamente os gradientes usando a regra da cadeia.

Mais a frente, o autograd faz:

* Executa a operação solicitada para calcular um tensor resultante
* Manter a função grandiente da operação no DAG

A passagem para trás começa quando ```.backward()``` é chamado na raiz DAG.

```autograd``` então:

* Calcula os gradientes de cada ```.grad_fn```
* ```.grad``` acumula-os no atributo do respectivo tensor
* usando a regra da cadeia, propaga-se até os tensores folha